<a href="https://colab.research.google.com/github/sashthamanik/deepfake-glab/blob/main/IITDelhi_Project_Deepfake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 Install Required Libraries

In [2]:
!pip install torch torchvision torchaudio
!pip install tensorflow keras opencv-python numpy dlib tqdm ffmpeg albumentations


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

 Extract Frames from Videos

In [26]:
import cv2
import os
#John.mp4
input_dir = "/content/drive/MyDrive/deepfake/input/"
output_dir = "/content/drive/MyDrive/deepfake/input/dataset/frames/"
os.makedirs(output_dir, exist_ok=True)

for video_name in os.listdir(input_dir):
    video_path = os.path.join(input_dir, video_name)
    cap = cv2.VideoCapture(video_path)
    count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_path = os.path.join(output_dir, f"{video_name}_frame_{count}.jpg")
        cv2.imwrite(frame_path, frame)
        count += 1

    cap.release()


Label Data (Real vs Fake)

In [28]:
import shutil
import os

real_dir = "/content/drive/MyDrive/deepfake/input/dataset/real/"
fake_dir = "/content/drive/MyDrive/deepfake/input/dataset/fake/"
os.makedirs(real_dir, exist_ok=True)
os.makedirs(fake_dir, exist_ok=True)

for img in os.listdir("/content/drive/MyDrive/deepfake/input/dataset/frames/"):
    if "fake" in img:
        shutil.move(f"/content/drive/MyDrive/deepfake/input/dataset/frames/{img}", fake_dir)
    else:
        shutil.move(f"/content/drive/MyDrive/deepfake/input/dataset/frames/{img}", real_dir)

Train the Deepfake Detection Model. We will use EfficientNet - CNN for classification. Load Data Using PyTorch

In [29]:
import cv2
import os
import shutil
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Subset
import torch
import numpy as np

# --- Extract Frames from Videos ---
input_dir = "/content/drive/MyDrive/deepfake/input/"
output_dir = "/content/drive/MyDrive/deepfake/input/dataset/frames/"
os.makedirs(output_dir, exist_ok=True)

for video_name in os.listdir(input_dir):
    video_path = os.path.join(input_dir, video_name)
    cap = cv2.VideoCapture(video_path)
    count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_path = os.path.join(output_dir, f"{video_name}_frame_{count}.jpg")
        cv2.imwrite(frame_path, frame)
        count += 1

    cap.release()

# --- Label Data (Real vs Fake) ---
# Instead of moving the images, we'll create a list of image paths and their labels
all_images = []
all_labels = []

for img in os.listdir(output_dir):
    image_path = os.path.join(output_dir, img)
    if "fake" in img:
        all_labels.append(1)  # 1 for fake
    else:
        all_labels.append(0)  # 0 for real
    all_images.append(image_path)

# --- Train the Deepfake Detection Model ---
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# Create a custom dataset class
class DeepfakeDataset(torch.utils.data.Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Convert to RGB

        if self.transform:
            image = self.transform(image)

        label = self.labels[idx]
        return image, label


# Create the dataset
dataset = DeepfakeDataset(all_images, all_labels, transform=transform)

# Split into train and validation sets (optional but recommended)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
#val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False) # Use if you split into validation set

Define the Model (EfficientNet)

In [30]:
import torch.nn as nn
import torchvision.models as models

class DeepfakeDetector(nn.Module):
    def __init__(self):
        super(DeepfakeDetector, self).__init__()
        self.model = models.efficientnet_b0(pretrained=True)
        self.model.classifier = nn.Sequential(
            nn.Linear(1280, 512),
            nn.ReLU(),
            nn.Linear(512, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

model = DeepfakeDetector()


Train the Model

In [31]:
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

# Define transformations
transform = transforms.Compose([
    transforms.ToPILImage(),  # Convert NumPy array to PIL Image if needed
    transforms.Resize((224, 224)),
    transforms.ToTensor(),  # Convert PIL Image to PyTorch tensor
    transforms.Normalize([0.5], [0.5])
])

# Load dataset correctly
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = cv2.imread(self.image_paths[idx])  # Read image as NumPy array
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
        label = self.labels[idx]

        if self.transform:
            img = self.transform(img)  # Apply transformations

        return img, torch.tensor(label, dtype=torch.float32)  # Ensure label is tensor

# Example dataset setup
image_paths = ["/content/drive/MyDrive/deepfake/input/dataset/real/", "/content/drive/MyDrive/deepfake/input/dataset/fake/"]  # Replace with actual paths
labels = [0, 1]  # 0 for real, 1 for fake

dataset = CustomDataset(image_paths, labels, transform=transform)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)


Save the Model

In [32]:
torch.save(model.state_dict(), "/content/drive/MyDrive/deepfake/deepfake_detector.pth")

Test Deepfake Detection

In [35]:
from PIL import Image
import torch
from torchvision import transforms

# Define transformation (Make sure to exclude ToPILImage here)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),  # Converts PIL image directly to tensor
    transforms.Normalize([0.5], [0.5])
])

def detect_deepfake(image_path):
    image = Image.open(image_path).convert("RGB")  # Ensure 3-channel RGB
    image = transform(image).unsqueeze(0)  # Apply transformations and add batch dimension

    model.eval()
    with torch.no_grad():
        output = model(image)
        return "fake" if output.item() > 0.5 else "real"

# Test the function with an image path
print(detect_deepfake("/content/drive/MyDrive/deepfake/input/dataset/frames/John.mp4_frame_0.jpg"))



fake
